In [7]:
# coding=utf-8
"""
This module contains the tests for the exchanges classes
"""
from fastlane_bot import Bot, Config
from fastlane_bot.bot import CarbonBot
from fastlane_bot.tools.cpc import ConstantProductCurve
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC
from fastlane_bot.events.exchanges import UniswapV2, UniswapV3, SushiswapV2, CarbonV1, BancorV3
from fastlane_bot.events.interface import QueryInterface
from fastlane_bot.helpers.poolandtokens import PoolAndTokens
from fastlane_bot.helpers import TradeInstruction, TxReceiptHandler, TxRouteHandler, TxSubmitHandler, TxHelpers, TxHelper
from fastlane_bot.events.managers.manager import Manager
from fastlane_bot.events.interface import QueryInterface
from joblib import Parallel, delayed
import pytest
import math
import json
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV3))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(SushiswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonV1))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(BancorV3))
from fastlane_bot.testing import *
from fastlane_bot.modes import triangle_single_bancor3
#plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

ConstantProductCurve v3.2 (15/Sep/2023)
CarbonBot v3-b2.2 (20/June/2023)
UniswapV2 v0.0.2 (2023-08-27)
UniswapV3 v0.0.2 (2023-08-27)
SushiswapV2 v0.0.2 (2023-08-27)
CarbonV1 v0.0.2 (2023-08-27)
BancorV3 v0.0.2 (2023-08-27)
Version = 3-b2.2 [requirements >= 3.0 is met]


# Single Mode [NB038]

In [8]:
C = cfg = Config.new(config=Config.CONFIG_MAINNET)
C.DEFAULT_MIN_PROFIT_BNT = 0.02
C.DEFAULT_MIN_PROFIT = 0.02
cfg.DEFAULT_MIN_PROFIT_BNT = 0.02
cfg.DEFAULT_MIN_PROFIT = 0.02
assert (C.NETWORK == C.NETWORK_MAINNET)
assert (C.PROVIDER == C.PROVIDER_ALCHEMY)
setup_bot = CarbonBot(ConfigObj=C)
pools = None
with open('fastlane_bot/data/tests/latest_pool_data_testing.json') as f:
    pools = json.load(f)
pools = [pool for pool in pools]
pools[0]
static_pools = pools
state = pools
exchanges = list({ex['exchange_name'] for ex in state})
db = QueryInterface(state=state, ConfigObj=C, exchanges=exchanges)
setup_bot.db = db

static_pool_data_filename = "static_pool_data"

static_pool_data = pd.read_csv(f"fastlane_bot/data/{static_pool_data_filename}.csv", low_memory=False)
    
uniswap_v2_event_mappings = pd.read_csv("fastlane_bot/data/uniswap_v2_event_mappings.csv", low_memory=False)
        
tokens = pd.read_csv("fastlane_bot/data/tokens.csv", low_memory=False)
        
exchanges = "carbon_v1,bancor_v3,uniswap_v3,uniswap_v2,sushiswap_v2"

exchanges = exchanges.split(",")


alchemy_max_block_fetch = 20
static_pool_data["cid"] = [
        cfg.w3.keccak(text=f"{row['descr']}").hex()
        for index, row in static_pool_data.iterrows()
    ]
# Filter out pools that are not in the supported exchanges
static_pool_data = [
    row for index, row in static_pool_data.iterrows()
    if row["exchange_name"] in exchanges
]

static_pool_data = pd.DataFrame(static_pool_data)
static_pool_data['exchange_name'].unique()
# Initialize data fetch manager
mgr = Manager(
    web3=cfg.w3,
    cfg=cfg,
    pool_data=static_pool_data.to_dict(orient="records"),
    SUPPORTED_EXCHANGES=exchanges,
    alchemy_max_block_fetch=alchemy_max_block_fetch,
    uniswap_v2_event_mappings=uniswap_v2_event_mappings,
    tokens=tokens.to_dict(orient="records"),
)

# Add initial pools for each row in the static_pool_data
start_time = time.time()
Parallel(n_jobs=-1, backend="threading")(
    delayed(mgr.add_pool_to_exchange)(row) for row in mgr.pool_data
)
cfg.logger.info(f"Time taken to add initial pools: {time.time() - start_time}")

# check if any duplicate cid's exist in the pool data
mgr.deduplicate_pool_data()
cids = [pool["cid"] for pool in mgr.pool_data]
assert len(cids) == len(set(cids)), "duplicate cid's exist in the pool data"
def init_bot(mgr: Manager) -> CarbonBot:
    """
    Initializes the bot.

    Parameters
    ----------
    mgr : Manager
        The manager object.

    Returns
    -------
    CarbonBot
        The bot object.
    """
    mgr.cfg.logger.info("Initializing the bot...")
    bot = CarbonBot(ConfigObj=mgr.cfg)
    bot.db = db
    bot.db.mgr = mgr
    assert isinstance(
        bot.db, QueryInterface
    ), "QueryInterface not initialized correctly"
    return bot
bot = init_bot(mgr)
# add data cleanup steps from main.py
bot.db.handle_token_key_cleanup()
bot.db.remove_unmapped_uniswap_v2_pools()
bot.db.remove_zero_liquidity_pools()
bot.db.remove_unsupported_exchanges()
tokens = bot.db.get_tokens()
ADDRDEC = {t.key: (t.address, int(t.decimals)) for t in tokens if not math.isnan(t.decimals)}
flashloan_tokens = bot.setup_flashloan_tokens(None)
CCm = bot.setup_CCm(None)
pools = db.get_pool_data_with_tokens()

arb_mode = "single"

2023-09-18 11:18:48,713 [fastlane:INFO] - 
2023-09-18 11:18:48,713 [fastlane:INFO] - 
2023-09-18 11:18:48,713 [fastlane:INFO] - **********************************************
2023-09-18 11:18:48,713 [fastlane:INFO] - **********************************************
2023-09-18 11:18:48,714 [fastlane:INFO] - The logging path is set to: logs/20230918-111848/bot.log
2023-09-18 11:18:48,714 [fastlane:INFO] - The logging path is set to: logs/20230918-111848/bot.log
2023-09-18 11:18:48,715 [fastlane:INFO] - **********************************************
2023-09-18 11:18:48,715 [fastlane:INFO] - **********************************************
2023-09-18 11:18:48,716 [fastlane:INFO] - 
2023-09-18 11:18:48,716 [fastlane:INFO] - 
2023-09-18 11:18:50,035 [fastlane:INFO] - Retrieved 168 carbon pairs from contract
2023-09-18 11:18:50,035 [fastlane:INFO] - Retrieved 168 carbon pairs from contract
2023-09-18 11:18:50,957 [fastlane:INFO] - Time taken to add initial pools: 0.06984496116638184
2023-09-18 11

In [9]:
assert(cfg.DEFAULT_MIN_PROFIT_BNT <= 0.02), f"[TestSingleMode], DEFAULT_MIN_PROFIT_BNT must be <= 0.02 for this Notebook to run, currently set to {cfg.DEFAULT_MIN_PROFIT_BNT}"
assert(C.DEFAULT_MIN_PROFIT_BNT <= 0.02), f"[TestSingleMode], DEFAULT_MIN_PROFIT_BNT must be <= 0.02 for this Notebook to run, currently set to {cfg.DEFAULT_MIN_PROFIT_BNT}"

## Test_arb_mode_class

In [10]:
arb_finder = bot._get_arb_finder("single")
assert arb_finder.__name__ == "FindArbitrageSinglePairwise", f"[TestSingleMode] Expected arb_finder class name name = FindArbitrageSinglePairwise, found {arb_finder.__name__}"

## Test_tokens_and_combos

In [11]:
arb_finder = bot._get_arb_finder("single")
finder2 = arb_finder(
            flashloan_tokens=flashloan_tokens,
            CCm=CCm,
            mode="bothin",
            result=bot.AO_TOKENS,
            ConfigObj=bot.ConfigObj,
        )
all_tokens, combos = finder2.find_arbitrage()
assert len(all_tokens) == 545, f"[TestMultiMode] Using wrong dataset, expected 545 tokens, found {len(all_tokens)}"
assert len(combos) == 3264, f"[TestMultiMode] Using wrong dataset, expected 3264 tokens, found {len(combos)}"

### Test_Single_Arb_Finder_vs_run

In [12]:
run_full = bot._run(flashloan_tokens=flashloan_tokens, CCm=CCm, arb_mode=arb_mode, data_validator=False, result=bot.XS_ARBOPPS)
arb_finder = bot._get_arb_finder("single")
finder = arb_finder(
            flashloan_tokens=flashloan_tokens,
            CCm=CCm,
            mode="bothin",
            result=bot.AO_CANDIDATES,
            ConfigObj=bot.ConfigObj,
        )
r = finder.find_arbitrage()
assert len(r) == 26, f"[TestSingleMode] Expected 26 arbs, found {len(r)}"
assert len(r) == len(run_full), f"[TestSingleMode] Expected arbs from .find_arbitrage - {len(r)} - to match _run - {len(run_full)}"

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

[PairOptimizer.optimize] curves_t agEUR-Bce8/USDC-eB48
[PairOptimizer.optimize] p_optimal OptimizerBase.SimpleResult(result=1.1012586819982886, method='bisection', errormsg=None, context_dct=None) full_result [-0.00028294  0.00031158]
[PairOptimizer.optimize] curves_t agEUR-Bce8/USDC-eB48
[PairOptimizer.optimize] p_optimal OptimizerBase.SimpleResult(result=1.1012068414056138, method='bisection', errormsg=None, context_dct=None) full_result [ 1.56512065e-03 -1.18761945e+01]
[PairOptimizer.optimize] curves_t agEUR-Bce8/USDC-eB48
[PairOptimizer.optimize] p_optimal OptimizerBase.SimpleResult(result=1.0977629060087901, method='bisection', errormsg=None, context_dct=None) full_result [ 2.36049527e-06 -2.59128865e-06]
[PairOptimizer.optimize] curves_t agEUR-Bce8/USDC-eB48
[PairOptimizer.optimize] p_optimal OptimizerBase.SimpleResult(result=1.095424874858844, method='bisection', errormsg=None, context_dct=None) full_result [-1.56286405e-05 -2.33128487e-01]


...

[PairOptimizer.optimize] curves_t SNX-2a6F/WETH-6Cc2
[PairOptimizer.optimize] p_optimal OptimizerBase.SimpleResult(result=0.0010930027501734703, method='bisection', errormsg=None, context_dct=None) full_result [ 8.45163665e-03 -9.23766231e-06]
[PairOptimizer.optimize] curves_t SNX-2a6F/WETH-6Cc2
[PairOptimizer.optimize] p_optimal OptimizerBase.SimpleResult(result=0.00109300277135908, method='bisection', errormsg=None, context_dct=None) full_result [ 1.19889504e-04 -1.31039656e-07]
[PairOptimizer.optimize] curves_t SNX-2a6F/WETH-6Cc2
[PairOptimizer.optimize] p_optimal OptimizerBase.SimpleResult(result=0.0010827174934704142, method='bisection', errormsg=None, context_dct=None) full_result [-1.71903921e-05  1.86123383e-08]
[PairOptimizer.optimize] curves_t SNX-2a6F/WETH-6Cc2
[PairOptimizer.optimize] p_optimal OptimizerBase.SimpleResult(result=0.0010827174789443481, method='bisection', errormsg=None, context_dct=None) full_result [ 2.19765025e-05 -2.37943425e-08]
[PairOptimizer.optimize] c

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [13]:
r

[(0.5926912321210337,
                                                      agEUR-Bce8   USDC-eB48
  0x9546a8b47738f2e0584eca2046653b7b16dc9474296a7...  119.967337 -131.637999
  21778071482940061661655974875633165533369-1        -119.967353  131.404870
  PRICE                                                 1.095425    1.000000
  AMMIn                                               119.967337  131.404870
  AMMOut                                             -119.967353 -131.637999
  TOTAL NET                                            -0.000016   -0.233128,
  ({'cid': '0x9546a8b47738f2e0584eca2046653b7b16dc9474296a73d548888c6dae78759d',
    'tknin': 'agEUR-Bce8',
    'amtin': 119.96733726689126,
    'tknout': 'USDC-eB48',
    'amtout': -131.6379988080007,
    'error': None},
   {'cid': '21778071482940061661655974875633165533369-1',
    'tknin': 'USDC-eB48',
    'amtin': 131.40487032127567,
    'tknout': 'agEUR-Bce8',
    'amtout': -119.9673528955318,
    'error': None}),
  'USDC-eB48',
 

## Test_no_multi_carbon

In [14]:
arb_finder = bot._get_arb_finder("single")
finder = arb_finder(
            flashloan_tokens=flashloan_tokens,
            CCm=CCm,
            mode="bothin",
            result=bot.AO_CANDIDATES,
            ConfigObj=bot.ConfigObj,
        )
r = finder.find_arbitrage()
multi_carbon_count = 0

for arb in r:
    (
            best_profit,
            best_trade_instructions_df,
            best_trade_instructions_dic,
            best_src_token,
            best_trade_instructions,
        ) = arb
    if len(best_trade_instructions_dic) > 2:
        multi_carbon_count += 1

assert multi_carbon_count == 0, f"[TestSingleMode] Expected arbs without multiple Carbon curves, but found {len(multi_carbon_count)}"

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

[PairOptimizer.optimize] curves_t agEUR-Bce8/USDC-eB48
[PairOptimizer.optimize] p_optimal OptimizerBase.SimpleResult(result=1.1012586819982886, method='bisection', errormsg=None, context_dct=None) full_result [-0.00028294  0.00031158]
[PairOptimizer.optimize] curves_t agEUR-Bce8/USDC-eB48
[PairOptimizer.optimize] p_optimal OptimizerBase.SimpleResult(result=1.1012068414056138, method='bisection', errormsg=None, context_dct=None) full_result [ 1.56512065e-03 -1.18761945e+01]
[PairOptimizer.optimize] curves_t agEUR-Bce8/USDC-eB48
[PairOptimizer.optimize] p_optimal OptimizerBase.SimpleResult(result=1.0977629060087901, method='bisection', errormsg=None, context_dct=None) full_result [ 2.36049527e-06 -2.59128865e-06]
[PairOptimizer.optimize] curves_t agEUR-Bce8/USDC-eB48
[PairOptimizer.optimize] p_optimal OptimizerBase.SimpleResult(result=1.095424874858844, method='bisection', errormsg=None, context_dct=None) full_result [-1.56286405e-05 -2.33128487e-01]


...

[PairOptimizer.optimize] curves_t SNX-2a6F/WETH-6Cc2
[PairOptimizer.optimize] p_optimal OptimizerBase.SimpleResult(result=0.0010930027501734703, method='bisection', errormsg=None, context_dct=None) full_result [ 8.45163665e-03 -9.23766231e-06]
[PairOptimizer.optimize] curves_t SNX-2a6F/WETH-6Cc2
[PairOptimizer.optimize] p_optimal OptimizerBase.SimpleResult(result=0.00109300277135908, method='bisection', errormsg=None, context_dct=None) full_result [ 1.19889504e-04 -1.31039656e-07]
[PairOptimizer.optimize] curves_t SNX-2a6F/WETH-6Cc2
[PairOptimizer.optimize] p_optimal OptimizerBase.SimpleResult(result=0.0010827174934704142, method='bisection', errormsg=None, context_dct=None) full_result [-1.71903921e-05  1.86123383e-08]
[PairOptimizer.optimize] curves_t SNX-2a6F/WETH-6Cc2
[PairOptimizer.optimize] p_optimal OptimizerBase.SimpleResult(result=0.0010827174789443481, method='bisection', errormsg=None, context_dct=None) full_result [ 2.19765025e-05 -2.37943425e-08]
[PairOptimizer.optimize] c

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]